In [ ]:
import gymenv_v2
from gymenv_v2 import make_multiple_env
import numpy as np
from config import custom_config, easy_config, hard_config
from attention_only import Embedding


import wandb
wandb.login()
run=wandb.init(project="finalproject", entity="orcs4529", tags=["training-easy"])

%load_ext autoreload
%autoreload 2

In [2]:
env = make_multiple_env(**easy_config) 

loading training instances, dir instances/train_10_n60_m60 idx 0
loading training instances, dir instances/train_10_n60_m60 idx 1
loading training instances, dir instances/train_10_n60_m60 idx 2
loading training instances, dir instances/train_10_n60_m60 idx 3
loading training instances, dir instances/train_10_n60_m60 idx 4
loading training instances, dir instances/train_10_n60_m60 idx 5
loading training instances, dir instances/train_10_n60_m60 idx 6
loading training instances, dir instances/train_10_n60_m60 idx 7
loading training instances, dir instances/train_10_n60_m60 idx 8
loading training instances, dir instances/train_10_n60_m60 idx 9


In [3]:
import tensorflow as tf

In [4]:
s = env.reset()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-13


In [10]:
cons = np.concatenate((s[0], s[1].reshape(-1, 1)), axis=1).astype(float)

In [11]:
cuts = np.concatenate((s[3], s[4].reshape(-1, 1)), axis=1).astype(float)

In [12]:
cons_embedding = Embedding([10, 10], ['tanh', 'tanh'])
cuts_embedding = Embedding([10, 10], ['tanh', 'tanh'])

In [17]:
h = cons_embedding(cons)
g = cuts_embedding(cuts)

In [20]:
h.shape, g.shape

(TensorShape([60, 10]), TensorShape([61, 10]))

In [31]:
tf.nn.softmax(tf.reduce_mean(tf.matmul(g, tf.transpose(h)), axis=1))

[autoreload of attention_only failed: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 450, in superreload
    update_generic(old_obj, new_obj)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 387, in update_generic
    update(a, b)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 357, in update_class
    update_instances(old, new)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 312, in update_instances
    update_instances(old, new, obj.__dict__, visited)
  File "/L

<tf.Tensor: shape=(61,), dtype=float32, numpy=
array([0.01639344, 0.01639344, 0.01639344, 0.01639344, 0.01639344,
       0.01639344, 0.01639344, 0.01639344, 0.01639344, 0.01639344,
       0.01639344, 0.01639344, 0.01639344, 0.01639344, 0.01639344,
       0.01639344, 0.01639344, 0.01639344, 0.01639344, 0.01639344,
       0.01639344, 0.01639344, 0.01639344, 0.01639344, 0.01639344,
       0.01639344, 0.01639344, 0.01639344, 0.01639344, 0.01639344,
       0.01639344, 0.01639344, 0.01639344, 0.01639344, 0.01639344,
       0.01639344, 0.01639344, 0.01639344, 0.01639344, 0.01639344,
       0.01639344, 0.01639344, 0.01639344, 0.01639344, 0.01639344,
       0.01639344, 0.01639344, 0.01639344, 0.01639344, 0.01639344,
       0.01639344, 0.01639344, 0.01639344, 0.01639344, 0.01639344,
       0.01639344, 0.01639344, 0.01639344, 0.01639344, 0.01639344,
       0.01639345], dtype=float32)>

In [33]:
from attention_only import AttentionLayer

In [34]:
layer = AttentionLayer([10, 10], ['tanh', 'tanh'])

In [35]:
layer([cons, cuts])

ValueError: Can't convert non-rectangular Python sequence to Tensor.

In [27]:
tf.matmul(g[:1], tf.transpose(h[:1]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[6.050974]], dtype=float32)>

In [20]:
[x.shape for x in s]

[(60, 60), (60,), (60,), (61, 60), (61,)]

In [13]:
arr = np.concatenate((s[0], s[1].reshape(-1, 1)), axis=1).astype(float)

In [16]:
model(arr).shape

TensorShape([60, 10])

(60, 61)

### Run Parameters

In [ ]:
# %%wandb
#remove above line if you do not want to see inline plots from wandb

# hyper-parameters
lr = 1e-3  # learning rate for gradient update
batchsize = 128  # batchsize for buffer sampling
maxlength = 1000  # max number of tuples held by buffer
envname = "CartPole-v0"  # environment name
tau = 100  # time steps for target update
episodes = 500  # number of episodes to run
initialsize = 500  # initial time steps before start training
epsilon = 0.5  # constant for exploration
epsilon_decay = 0.99
gamma = .99  # discount

# initialize environment
env = gym.make(envname)
obssize = env.observation_space.low.size
actsize = env.action_space.n

ffn_params = [(50, 'relu'), (100, 'relu'), (actsize, 'linear')]

# initialize Q-function networks (princpal and target)
Qprincipal = Qfunction(obssize, actsize, lr, ffn_params)
Qtarget = Qfunction(obssize, actsize, lr, ffn_params)

# initialization of graph and buffer
buffer = ReplayBuffer(maxlength)

# main iteration
rrecord = []
totalstep = 0

for episode in range(episodes):
    
    obs = env.reset()
    done = False
    rsum = 0

    while not done:
        #greedy choice below. Use epsilon greedy for exploration
        if np.random.rand() < (1-epsilon):
          action = Qtarget.compute_argmaxQ(np.expand_dims(obs,0))
        else:
          action = env.action_space.sample()
        
        newobs, r, done, _ = env.step(action)
        done_ = 1 if done else 0
        e = (obs, action, r, done_, newobs)

        #IF NOT USING BUFFER:
        #use single sample (obs, action, r, done_, newobs) with Qtarget to compute target and train Qprincipal
        buffer.append(e)
        while buffer.number > maxlength:
            buffer.pop()

        if buffer.number > batchsize:
          batch = buffer.sample(batchsize)
          states, actions, rewards, dones, next_states = [np.array(x) for x in list(map(list, zip(*batch)))]
          done_idxs = np.where(dones)[0]
          max_qvalues = Qtarget.compute_maxQvalues(next_states)
          targets = rewards + gamma*max_qvalues
          targets[done_idxs] = rewards[done_idxs]
          Qprincipal.train(states, actions, targets)
        
        if totalstep % tau == 0:
          run_target_update(Qprincipal, Qtarget)
          
        # update
        totalstep += 1
        rsum += r
        obs = newobs 

    epsilon = max(epsilon*epsilon_decay, 0.01)
    #Below are for printing and debugging at the end of episode
    
    rrecord.append(rsum)
    if episode % 10 == 0:
        print('buffersize {}'.format(buffer.number))
        print('iteration {} ave training returns {}'.format(episode, np.mean(rrecord[-10:])))
        
    #printing moving averages for smoothed visualization. 
    fixedWindow=100
    movingAverage=0
    if len(rrecord) >= fixedWindow:
        movingAverage=np.mean(rrecord[len(rrecord)-fixedWindow:len(rrecord)-1])
    wandb.log({ "training reward" : rsum, "train reward moving average" : movingAverage})

In [ ]:
episodes

In [ ]:
for e in range(20):
    # gym loop
    s = env.reset()   # samples a random instance every time env.reset() is called
    d = False
    t = 0
    repisode = 0

    while not d:
        #Take a random action
        a = np.random.randint(0, s[-1].size, 1)            # s[-1].size shows the number of actions, i.e., cuts available at state s

        #simulate the environment to get the next state
        s, r, d, _ = env.step(list(a))
        print('episode', e, 'step', t, 'reward', r, 'action space size', s[-1].size, 'action', a[0])

        A, b, c0, cuts_a, cuts_b = s
        #print(A.shape, b.shape, c0.shape, cuts_a.shape, cuts_b.shape)

        t += 1
        repisode += r

        #wandb logging
        wandb.log({"Training reward (easy config)" : repisode})
    #make sure to use the correct tag in wandb.init in the initialization on top


